# Working with surftemp-sst data


## Tutorial 1 - Getting Started

This tutorial helps you get started with the Sea Surface Temperature Daily Analysis dataset: European Space Agency Climate Change Initiative product version 2.1, hosted by AWS opendata.

A brief summary of this dataset is:

> Global daily-mean sea surface temperatures, presented on a 0.05° latitude-longitude grid, with gaps between available daily observations filled by statistical means, spanning late 1981 to recent time.
>
> Synthesised from multiple Earth orbiting satellites carrying infrared imagers from thousands of billions of individual measurements. Underlying observation resolution ranges from 1 to 20 km, and after gap filling the typical feature resolution is ~20 km. Suitable for large-scale oceanographic meteorological and climatological applications, such as evaluating or constraining environmental models or case-studies of marine heat wave events.
>
> Adhering to community data standards and names. Includes temperature uncertainty information and auxiliary information about land-sea fraction and sea-ice coverage. To understand the data for your application, read the paper [1] using <a href="www.nature.com/articles/s41597-019-0236-x">www.nature.com/articles/s41597-019-0236-x</a> to cite in any published usage.
>
> The v2.1 record is known to have biases associated with desert dust aerosol and erratic calibration of early-record sensors [1]. Adjustments to reduce these biases and include additional uncertainty in these effects have been developed, as described in [2] and are applied to this data. These adjustments operate on monthly and >5 degree time-space scales.
> 
> [1] Merchant, C.J., Embury, O., Bulgin, C.E., Block, T., Corlett, G.K., Fiedler, E., Good, S.A., Mittaz, J., Rayner, N.A., Berry, D., Eastwood, S., Taylor, M., Tsushima, Y., Waterfall, A., Wilson, R. and Donlon, C. (2019), Satellite-based time-series of sea-surface temperature since 1981 for climate applications. Scientific Data 6, 223, doi:10.1038/s41597-019-0236-x
>
> [2] Merchant, C.J. and Embury, O. (2020) Adjusting for desert-dust-related biases in a climate data record of sea surface temperature. Remote Sensing, 12 (16). 2554. ISSN 2072-4292 doi:10.3390/rs12162554

Although the entire datset is over 300Gb in size, a subset of the data can be downloaded selectively to inspect particular areas, locations or time periods.  This tutorial will provide some basic examples of how to do this using the [xarray library]( http://xarray.pydata.org/en/stable/)

### 1.0 Installing dependencies

If you don't already have an environment created, you can create one that will run this notebook. From your Anaconda Prompt (windows) or bash (mac), change to the directory you have cloned the repository to and type: `conda env create -f environment.yml` then activate the env `conda activate sstenv`.


In [ ]:
import fsspec
import matplotlib
import s3fs
import xarray as xr

### 1.1 Opening the dataset

The following code will load the metadata for the dataset and display the dataset information. It is really fast because it uses 'lazy loading'. Data is only loaded when it is needed, and only the chunks that are needed are accessed.

In [ ]:
%%time
file_location = "s3://surftemp-sst/data/sst.zarr"

ikey = fsspec.get_mapper(file_location, anon=True)

ds = xr.open_zarr(ikey)

### 1.2 Inspecting the dataset

Xarray will print out all the variables in the dataset so you can explore them. Click on the icons to the right of the data to examine the data variable attributes and see some values for the data variable. At the bottom you can explore the global attributes for the dataset. This dataset contains:

| Variable name         | Description                                              | Units   |
| --------------------- | -------------------------------------------------------- | ------- |
| analysed_sst          | Analysed sea surface temperature (Kelvin)                | kelvin  |
| analysis_uncertainty  | Estimated error standard deviation of analysed_sst       | kelvin  |
| sea_ice_fraction      | The estimated fraction of the area covered by sea ice    | -       |
| mask                  | Bit mask (bit0:sea,bit1:land:bit2:lake,bit3:ice)         | -       |


In [ ]:
ds

### 1.3 Plot a timeseries of the `analyzed_sst` over a specific time range

The data covers the earth's surface at a resolution of 0.05 degrees.  If you are interested in temperatures at a single location, extract a timeseries at a specific location. This is easiest to do with two .sel calls. First we select our time range. Next we select the latitude and longitude specifing `method="nearest"` to find the nearest point.

In [ ]:
%%time
(
    ds["analysed_sst"]
    .sel(time=slice("1982-01-01", "1985-12-31"))
    .sel(lat=51, lon=-38, method="nearest")
).plot(figsize=(10, 5))

### 1.4 Plot data for a specific region

it is easy to select a subset of the data for  given range of latitudes and longitudes.  For example, to see the data for a region (the gulf of guinea) at a single point in time:

In [ ]:
%%time
(
    ds["analysed_sst"]
    .sel(time="2006-02-23")
    .sel(lat=slice(-5, 10), lon=slice(-5,10))
).plot(cmap="coolwarm", figsize=(5, 5))


NameError: name 'ds' is not defined

In [ ]:
ssts_ts.plot(cmap="coolwarm", figsize=(10, 10))

In [ ]:
### 1.5 Plotting global sea surface temperatures at a point in time

In [ ]:
global_ssts = sst_ds["analysed_sst"].sel(time="1983-01-01").load()
global_ssts.plot(cmap="coolwarm", figsize=(10, 5))

### 1.6 Inspecting other variables in the dataset

fetch the SST uncertainty data for a region (the gulf of guinea) at a single point in time:

In [ ]:
sst_ds["analysed_sst_uncertainty"].sel(time="2006-02-23").sel(
    lat=slice(-5, 10), lon=slice(-5, 10)
).plot()

plot global sea ice fraction on the first day in 1983

In [ ]:
global_sea_ice_fraction = sst_ds["sea_ice_fraction"].sel(time="1983-01-01")
global_sea_ice_fraction.plot(figsize=(10, 5))

Plot the mask over Scandanavia

In [ ]:
sst_ds["mask"].sel(time="2006-01-18").sel(lon=slice(2, 42), lat=slice(53, 72)).plot()